In [1]:
import os
import lightgbm as lgb
import json
import pandas as pd
import nltk
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.feature_extraction.text import TfidfVectorizer as TV
from sklearn.model_selection import train_test_split
import numpy as np
import pickle
np.random.seed(109)

In [2]:
os.getcwd()

'C:\\Users\\tlebr\\Google Drive\\fdd\\dailynews\\cybernews\\notebooks'

In [30]:
os.chdir(r"C:\Users\tlebr\Google Drive\fdd\dailynews\cybernews")
p16 = r"data\jsons\2016.json"
p17 = r"data\jsons\2017.json"
p18 = r"data\jsons\2018.json"
p19 = r"data\jsons\2019.json"
p20 = r"data\jsons\2020.json"
pf = r"data\jsons\Feb.json"
pmar = r"data\jsons\March.json"
pa = r"data\jsons\April.json"
pmay = r"data\jsons\May.json"

js_ls = []
for file in [p16, p17, p18, p19, p20, pf, pmar, pa, pmay]:
    with open(file, encoding="utf-8") as fp:
        js = json.load(fp)
        for el in js:
            js_ls.append(el)
df = pd.json_normalize(js_ls)

In [35]:
df.ranking = abs(df.ranking - max(df.ranking)) + 1

In [36]:
df.head()

,title,url,ranking,filename,body,date,source,author
0,"Second hacker group targets SWIFT users, Syman...",https://finance.yahoo.com/news/second-hacker-g...,15,Daily_Cyber_Briefing_October13.docx,Cyber-security firm Symantec Corp said on Tues...,2021-10-11 00:00:00,Reuters,[By Jim Finkle]
1,IAEA chief: Nuclear power plant was disrupted ...,http://www.reuters.com/article/us-nuclear-cybe...,14,Daily_Cyber_Briefing_October13.docx,A nuclear power plant became the target of a d...,2021-10-10 00:00:00,Reuters,[By Andrea Shalal]
2,How France's TV5 was almost destroyed by 'Russ...,http://www.bbc.com/news/technology-37590375,13,Daily_Cyber_Briefing_October13.docx,TV5Monde was taken off air in April 2015. A gr...,2021-10-10 00:00:00,BBC News,[By Gordon Corera]
3,Bureau of Meteorology hacked by foreign spies ...,http://www.abc.net.au/news/2016-10-12/bureau-o...,12,Daily_Cyber_Briefing_October13.docx,A foreign power managed to install malicious s...,2021-10-11 00:00:00,ABC (Australia),[By Andrew Greene]
4,How to Win the Cyberwar Against Russia,https://foreignpolicy.com/2016/10/12/how-to-wi...,11,Daily_Cyber_Briefing_October13.docx,Vladimir Putin’s brazen attack on U.S. democra...,2021-10-12 00:00:00,Foreign Policy,[By ]


In [37]:
files = pd.unique(df["filename"])
np.random.shuffle(files)
[train, val, test] = np.split(files, [int(.8 * len(files)), int(.9 * len(files))])


In [38]:
def match(row):
    if row.filename in train: 
        return "train"
    if row.filename in val:
        return "validation"
    if row.filename in test:
        return "test"
        

In [39]:
df["ttv"] = df.apply(lambda row: match(row), axis=1)

In [40]:
X_train = df[df['ttv'] == "train"]
X_val = df[df['ttv'] == "validation"]
X_test = df[df['ttv'] == "test"]
X_train = X_train.sort_values(by="filename")
X_val = X_val.sort_values(by="filename")
X_test = X_test.sort_values(by="filename")
y_train = X_train["ranking"]
y_val = X_val["ranking"]
y_test = X_test["ranking"]

In [41]:
train_group=X_train.groupby("filename").count()["title"].tolist()
val_group=X_val.groupby("filename").count()["title"].tolist()

In [42]:
stops = stopwords.words('english')
analyze = CV(stop_words=stops, min_df = 15, max_df=.5)
ps = PorterStemmer()
def stop_removal(text : str): 
    toks=analyze(text)
    return ' '.join([ps.stem(word) for word in toks if not ps.stem(word) in stops])

In [43]:
analyze = TV(stop_words=stops, min_df = 15, max_df=.5)
sparse_train = analyze.fit_transform(X_train["body"]).astype('float32')
sparse_val = analyze.transform(X_val["body"]).astype('float32')

In [44]:
gbm = lgb.LGBMRanker(max_depth=5)
gbm.fit(sparse_train, y_train, group=train_group, 
        eval_set=[(sparse_val, y_val)], eval_group = [val_group],
        early_stopping_rounds = 50)

[1]	valid_0's ndcg@1: 0.338023	valid_0's ndcg@2: 0.371917	valid_0's ndcg@3: 0.413844	valid_0's ndcg@4: 0.4708	valid_0's ndcg@5: 0.539535
Training until validation scores don't improve for 50 rounds
[2]	valid_0's ndcg@1: 0.34138	valid_0's ndcg@2: 0.410667	valid_0's ndcg@3: 0.469095	valid_0's ndcg@4: 0.52491	valid_0's ndcg@5: 0.573799
[3]	valid_0's ndcg@1: 0.376965	valid_0's ndcg@2: 0.446454	valid_0's ndcg@3: 0.492997	valid_0's ndcg@4: 0.539488	valid_0's ndcg@5: 0.575062
[4]	valid_0's ndcg@1: 0.379681	valid_0's ndcg@2: 0.429111	valid_0's ndcg@3: 0.489304	valid_0's ndcg@4: 0.548753	valid_0's ndcg@5: 0.598384
[5]	valid_0's ndcg@1: 0.413862	valid_0's ndcg@2: 0.45099	valid_0's ndcg@3: 0.516012	valid_0's ndcg@4: 0.561742	valid_0's ndcg@5: 0.612122
[6]	valid_0's ndcg@1: 0.418195	valid_0's ndcg@2: 0.442755	valid_0's ndcg@3: 0.506967	valid_0's ndcg@4: 0.558823	valid_0's ndcg@5: 0.605114
[7]	valid_0's ndcg@1: 0.435408	valid_0's ndcg@2: 0.454308	valid_0's ndcg@3: 0.521253	valid_0's ndcg@4: 0.56666

LGBMRanker(max_depth=5)

In [45]:
val_pred=gbm.predict(sparse_val)

In [46]:
len(gbm.evals_result_["valid_0"]["ndcg@1"])

95

In [47]:
X_val["pred"] = val_pred


In [48]:
gbm.best_score_

defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('ndcg@1', 0.4532914212469863),
                          ('ndcg@2', 0.5159141609060663),
                          ('ndcg@3', 0.5596603702170255),
                          ('ndcg@4', 0.5924122315259691),
                          ('ndcg@5', 0.649068281599732)])})

In [49]:
pickle.dump(analyze, open("models/analyze2.pickle", "wb"))
pickle.dump(gbm, open("models/gbm2.pickle", "wb"))
